In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatibl

In [ ]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.9 MB/s eta 0:00:00


In [ ]:
pip install tomotopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tomotopy as tp
tqdm.pandas()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
twitter_data = pd.read_csv("/content/drive/MyDrive/Grad School/Ukraine War Project/Data/Twitter_Data/all_tweets_test_2.csv")

<ipython-input-7-da990b9156c6>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  twitter_data = pd.read_csv("/content/drive/MyDrive/Grad School/Ukraine War Project/Data/Twitter_Data/all_tweets_test_2.csv")


In [ ]:
twitter_data = twitter_data.dropna(subset = ["text","created_at"])
twitter_data = twitter_data.drop_duplicates(subset='text', keep="first")

In [ ]:
twitter_data["Search_Key-Words"].value_counts()

Search_Key-Words
ukraine                33501
putin+war              32683
russia+war             28579
green+energy           27617
russia                 26550
ukraine+oil            21747
russia+invasion        20204
zelenskyy              18925
NATO                   17748
natural+gas            16227
ukraine+energy         14848
putin                  14673
putin+oil              11185
putin+energy            7010
war+energy              6583
zelenskyy+war           5472
stock+energy            2878
ukraine+stock           2513
russia+stock            2013
war+stock               1949
ukriane                 1458
putin+stock              476
kyiv+oil                 369
russia+natural+gas       168
kyiv+energy              163
ukraine+natural+gas       43
kyiv+stock                38
zelenskyy+energy          21
zelenskyy+oil             21
zelenskyy+stock            5
Name: count, dtype: int64

In [ ]:

import torch
device = torch.device("cuda")
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from torch.utils.data import DataLoader
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

#model.save_pretrained(MODEL)
text = '@John68Richmond @JaneSeyd @NorthShoreNews Thank goodness not all nato nations/cities think that way. Someone has to keep the deterrent alive. Otherwise Putins if the world might actually use tactical small yield weapons.'
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt').to(device) # this had to be coppied to the GPU for the model to work
output = model(**encoded_input)
scores = output[0][0].detach().cpu().numpy() #This is copying the results back to the CPU for output
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) negative 0.6195
2) neutral 0.3366
3) positive 0.044


In [ ]:
tokenized_data = twitter_data.copy()

tokenized_data["tokens"] = tokenized_data["text"].progress_apply(lambda x: tokenizer(x, return_tensors='pt'))

  0%|          | 0/315667 [00:00<?, ?it/s]

In [ ]:
#figure out how to batch load text data into GPU for much faster processing

def get_sentiment(text):
  
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt').to(device) #This is a vector for one tweet and I want to change it to an array with many tweets
  output = model(**encoded_input)
  scores = output[0][0].detach().cpu().numpy()
  scores = softmax(scores) # This would also output an array of scores of each of the tweets
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  sentiment = [-1,0, "Positive"]
  return (ranking[0]-1,scores[ranking[0]])


get_sentiment('@dcexaminer Because our dear leaders here in der Vest have gotten it right so many times before...\nhttps://t.co/5A48os1L9s\nhttps://t.co/BzzTx5Cf1g\nhttps://t.co/rfLwmdvKXy\n🤷\u200d♂️')

(1, 0.59412247)

In [ ]:
samp = twitter_data.sample(1)
print(samp["text"].head())

encoded_input = tokenizer(list(samp["text"]),return_tensors = "pt", padding =True,truncation=True)
encoded_input

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


230645    Germany's economy faces losses of nearly 400 b...
Name: text, dtype: object


{'input_ids': tensor([[    0, 24596,    18,   866,  2419,  2687,     9,   823,  3675,   325,
          2729,   149, 15294,   114,     5,   997,    11,  4174, 27196,  1626,
             8,    41, 21313,    15,  1083,  1007,  3315,     7,  5350,  4311,
            15,   476,  4898,     8,   539,     4, 38252,   115,  1355,    25,
           203,    25,   132,   207,    42,    76,     6,   309,     7,     5,
         27117,  5760,     4,  1205,   640,    90,     4,   876,    73,   330,
           500,   642,   534,   541, 32880,   406,   574,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:

samp_data = twitter_data.sample(5000)

samp_data["sentiment"] = samp_data["text"].progress_apply(get_sentiment)


  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
from google.colab import files
#samp_data.to_csv("samp_twitter_sentiment.csv", encoding = 'utf-8-sig')
#files.download('samp_twitter_sentiment.csv')

sentiment_data = pd.read_csv("/content/drive/MyDrive/Grad School/Ukraine War Project/Data/Twitter_Data/samp_twitter_sentiment-2.csv")

In [ ]:
sentiment_data['created_at_2'] = sentiment_data["created_at"].progress_apply(pd.to_datetime)

  0%|          | 0/80000 [00:00<?, ?it/s]

In [ ]:
from nltk.tokenize.casual import TweetTokenizer
tokenizer = TweetTokenizer(strip_handles=True, preserve_case=False)

In [ ]:
twitter_data["tokens"] = twitter_data['text'].progress_apply(tokenizer.tokenize)

In [ ]:
#from nltk.corpus import stopwords
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
from nltk.stem.porter import PorterStemmer
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

stop_words = stop_words + [".","(",")",",","/",":",";","&"]
#stop_words = list(stopwords.words('english')) + [".","!","(",")",",","/",":",";","&"]
#print(stop_words + [".","!","(",")",","])




for index, row_series in twitter_data.iterrows():
    twitter_data.at[index , "tokens"] = [p_stemmer.stem(word) for word in row_series["tokens"] if not word in stop_words]

In [ ]:
k = 40
min_df = 500
rm_top = 10
tw = tp.TermWeight.ONE
alpha = 0.1
eta = 0.01
tol = 1e-3


In [ ]:
%%time

mdl = tp.LDAModel(k=k, min_df=min_df, rm_top=rm_top, tw=tw, alpha=alpha, eta=eta)

for doc in twitter_data['tokens']:
    if doc:
        mdl.add_doc(doc)

last = np.NINF
for i in range(0, 5000, 50):
    mdl.train(50)
    ll = mdl.ll_per_word
    print(f'{i:5d} LL = {ll:7.4f}', flush=True)
    if ll - last < tol:
        break
    else:
        last = ll

print(f'Done!')

In [ ]:

for k in range(mdl.k):
    print(f'{k:3d} ', ', '.join(s for s,_ in mdl.get_topic_words(k)))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis

topic_term_dists = np.stack([mdl.get_topic_word_dist(k)
                             for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq
prepared_data = pyLDAvis.prepare(topic_term_dists,
                                 doc_topic_dists,
                                 doc_lengths,
                                 vocab,
                                 term_frequency, 
                                 mds='tsne', 
                                 sort_topics=False
                                 )

pyLDAvis.display(prepared_data)

In [ ]:
sentiment_data["tokens"] = sentiment_data['text'].progress_apply(tokenizer.tokenize)

In [ ]:
#from nltk.corpus import stopwords
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
from nltk.stem.porter import PorterStemmer
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

stop_words = stop_words + [".","(",")",",","/",":",";","&"]
#stop_words = list(stopwords.words('english')) + [".","!","(",")",",","/",":",";","&"]
#print(stop_words + [".","!","(",")",","])


for index, row_series in sentiment_data.iterrows():
    sentiment_data.at[index , "tokens"] = [word for word in row_series["tokens"] if not word in stop_words]

In [ ]:
%%time

mdl_sent = tp.LDAModel(k=k, min_df=min_df, rm_top=rm_top, tw=tw, alpha=alpha, eta=eta)

for doc in sentiment_data['tokens']:
    if doc:
        mdl.add_doc(doc)

last = np.NINF
for i in range(0, 5000, 50):
    mdl_sent.train(50)
    ll = mdl_sent.ll_per_word
    print(f'{i:5d} LL = {ll:7.4f}', flush=True)
    if ll - last < tol:
        break
    else:
        last = ll

print(f'Done!')

In [ ]:
for k in range(mdl_sent.k):
    print(f'{k:3d} ', ', '.join(s for s,_ in mdl_sent.get_topic_words(k)))

In [ ]:
topic_term_dists = np.stack([mdl_sent.get_topic_word_dist(k)
                             for k in range(mdl_sent.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl_sent.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl_sent.docs])
vocab = list(mdl_sent.used_vocabs)
term_frequency = mdl_sent.used_vocab_freq
prepared_data = pyLDAvis.prepare(topic_term_dists,
                                 doc_topic_dists,
                                 doc_lengths,
                                 vocab,
                                 term_frequency, 
                                 mds='tsne', 
                                 sort_topics=False
                                 )

pyLDAvis.display(prepared_data)

In [ ]:

mdl_load = tp.LDAModel.load("/content/drive/Shareddrives/Dylan/Data/ukraine_topics.bin")
for k in range(mdl_load.k):
    print(f'{k:3d} ', ', '.join(s for s,_ in mdl_load.get_topic_words(k)))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis

topic_term_dists = np.stack([mdl_load.get_topic_word_dist(k)
                             for k in range(mdl_load.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl_load.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl_load.docs])
vocab = list(mdl_load.used_vocabs)
term_frequency = mdl_load.used_vocab_freq
prepared_data = pyLDAvis.prepare(topic_term_dists,
                                 doc_topic_dists,
                                 doc_lengths,
                                 vocab,
                                 term_frequency, 
                                 mds='tsne', 
                                 sort_topics=False
                                 )

pyLDAvis.display(prepared_data)

In [ ]:
topics = pd.DataFrame({'label':np.nan,
                       'words':[', '.join(s for s,_ in mdl_load.get_topic_words(k)) 
                                for k in range(mdl_load.k)]},
                      index=range(mdl_load.k))
topics



In [ ]:
sentiment_data = sentiment_data[sentiment_data["tokens"].map(len) !=0]

In [ ]:
topics = pd.read_csv('/content/drive/Shareddrives/Dylan/Data/topics.csv')
topics

In [ ]:
sentiment_data['docs'] = [mdl_load.make_doc(words=toks) for toks in sentiment_data['tokens']]
topic_dist, ll = mdl_load.infer(sentiment_data['docs'])

In [ ]:
import re
sentiment_data["sentiment_value"] = sentiment_data["sentiment"].apply(lambda x: re.findall('-?\d+\.?\d*',x))
sentiment_data["sentiment_value"] = sentiment_data["sentiment_value"].apply(lambda x: float(x[0]) * float(x[1]))

In [ ]:
sentiment_data.drop(['Unnamed: 0.1', 'Unnamed: 0',],axis = 1,inplace =True)

In [ ]:
sentiment_data["topic"] = sentiment_data["docs"].progress_apply(lambda x: x.get_topics(top_n=1)[0])

In [ ]:

sentiment_data['topic_name'] = sentiment_data["topic"].apply(lambda x: topics["label"][x[0]])

In [ ]:
sentiment_data.head()

In [ ]:
sentiment_dict = {}
for topic in sentiment_data["topic_name"].unique():
    sentiment_dict[topic] = sentiment_data[sentiment_data["topic_name"] == topic]["sentiment_value"].mean()
    

sentiment_dict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize = (15,15))
plt.tight_layout()

plt.subplot(331)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "War"]["created_at_2"])
plt.title("Topic: War")
plt.xticks(rotation = 45)

plt.subplot(332)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Ukraine"]["created_at_2"])
plt.title("Topic: Ukraine")
plt.xticks(rotation = 45)

plt.subplot(333)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "NATO"]["created_at_2"])
plt.title("Topic: NATO")
plt.xticks(rotation = 45)

plt.subplot(334)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Energy"]["created_at_2"])
plt.title("Topic: Energy")
plt.xticks(rotation = 45)

plt.subplot(335)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Green Energy"]["created_at_2"])
plt.title("Topic: Green Energy")
plt.xticks(rotation = 45)

plt.subplot(336)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Stocks"]["created_at_2"])
plt.title("Topic: Stocks")
plt.xticks(rotation = 45)

plt.subplot(337)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Leaders"]["created_at_2"])
plt.title("Topic: Leaders")
plt.xticks(rotation = 45)

plt.subplot(338)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Hashtag"]["created_at_2"])
plt.title("Topic: Hashtag")
plt.xticks(rotation = 45)

plt.subplot(339)
plt.hist(sentiment_data[sentiment_data["topic_name"] == "Countries"]["created_at_2"])
plt.title("Topic: Countries")
plt.xticks(rotation = 45)

plt.show()